In [2]:
import pandas as pd
import numpy as np
# from utils import search_query

from sentence_transformers import SentenceTransformer
import faiss
import pickle
encoder = SentenceTransformer("all-mpnet-base-v2")

/Users/ohamugochukwu/Documents/Oham-Job-Board/jobsite/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/ohamugochukwu/Documents/Oham-Job-Board/jobsite/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
queryString = """
<div><strong>Join Our Team</strong></div><div> </div><div>Oowlish, one of Latin America's rapidly expanding software development companies, is seeking experienced technology professionals to enhance our diverse and vibrant team.</div><div> </div><div>As a valued member of Oowlish, you will collaborate with premier clients from the United States and Europe, contributing to pioneering digital solutions. Our commitment to creating a nurturing work environment is recognized by our certification as a Great Place to Work, where you will have opportunities for professional development, growth, and a chance to make a significant international impact.</div><div> </div><div>We offer the convenience of remote work, allowing you to craft a work-life balance that suits your personal and professional needs. We're looking for candidates who are passionate about technology, proficient in English, and excited to engage in remote collaboration for a worldwide presence.</div><div> </div><div> </div><div>Requirements</div><ul><li>Bachelor´s degree in marketing, business, statistics/analytics  or related field </li><li>3-5 years of work experience in marketing research , statistical or data analysis</li><li>Excellent digital aptitude, significant familiarity with ad ops technology and affiliate tracking and management tolos</li><li>Excellent organizational, communication, presentation and analytical skills</li><li>Experience with analytics systems (Google Analytics, Tableu and Excel), SQL and Data warehouses/data visualization </li><li>Demonstrated understanding of marketing and web Kpi´s (CTR, ROI, CVR, CAC, MAU)</li></ul><div> </div><ul><li> </li></ul><div> </div><div><strong>Benefits &amp; Perks:</strong></div><div> </div><div>Home office;</div><div>Flexible Hours</div><div>Competitive compensation based on experience;</div><div>Career plans to allow for extensive growth in the company;</div><div>International Projects;</div><div>Oowlish English Program (Technical and Conversational);</div><div>Oowlish Fitness with Total Pass;</div><div>Connecting You (Internet allowance);</div><div>Anniversary bonus;</div><div>Wedding gift;</div><div>Pet adoption incentive;</div><div>New baby Oowl bonus;</div><div>Back to School bonus;</div><div>Streaming Subscription;</div><div>PTO Bonus;</div><div>Games and Competitions;</div><div>Enjoy your national Holidays.</div><div> </div><div><strong>You can also apply here:</strong></div><div> </div><div>Website: https://www.oowlish.com/work-with-us/</div><div>LinkedIn: https://www.linkedin.com/company/oowlish/jobs/</div><div>Instagram: https://www.instagram.com/oowlishtechnology/</div><div> </div><div>#LI-LM1</div><div>#LI-CD1</div><div>#LI-EA1</div><div>#LI-TC1</div><div>#LI-ET1</div><div>#LI-TT1</div><div>#LI-JH1</div><div>#LI-DP1</div><div>#LI-LS1</div><div>#LI-AB1</div><div>#LI-KN1</div><div>#LI-SR1</div><div>#LI-JS1</div><div>#LI-FZ1</div><img/>
"""

In [5]:

# Load the FAISS index
# Deserialize the index using pickle
with open('jobIndex.pk1', 'rb') as f:
    jobIndex = pickle.load(f)
    print("Job vectors loaded")

with open('jobId.pk1', 'rb') as f:
    jobId = pickle.load(f)
    # print(jobId)
    print("Job ID loaded")

# Convert to a numpy array with int32 data type
# jobId = np.array(jobId, dtype=np.int32)

# Check if all elements are int32
all_int32 = all(isinstance(number, np.int64) for number in jobId)
# print(all_int32)

# Check the shape
# print(jobSearch_vec.shape)  # Should output: (1, 768)
index = faiss.IndexIDMap(faiss.IndexFlatL2(768))
index.add_with_ids(jobIndex, jobId)
print(index.ntotal)

# SimilarLookUpTable = faiss.IndexFlatL2(jobIndex.shape[1])
# SimilarLookUpTable.add(jobIndex)


# Encode the search query
jobSearch_vec = encoder.encode(queryString)

# Reshape the vector to match the expected dimensions
jobSearch_vec = jobSearch_vec.reshape((1, -1))  # Shape: (1, 768)
print(jobSearch_vec.shape)

# dis, Loca = SimilarLookUpTable.search(jobSearch_vec, 15)
distance, Location = index.search(jobSearch_vec, 10)

Job vectors loaded
Job ID loaded
3304
(1, 768)


In [11]:
with open('jobIndex.pk1', 'rb') as f:
    jobIndex = pickle.load(f)
    # print("Job vectors loaded")

jobIndex


array([[ 0.03359122, -0.01374388, -0.04224411, ...,  0.0454323 ,
         0.04199422, -0.03748216],
       [ 0.05418294,  0.02125891, -0.01160441, ...,  0.00384664,
         0.00583727, -0.04800911],
       [ 0.06379892,  0.00157726, -0.0260894 , ...,  0.00026244,
        -0.04051778, -0.02760356],
       ...,
       [ 0.03382177,  0.07250165, -0.02072722, ...,  0.03050919,
         0.0002706 , -0.01424059],
       [ 0.04952749,  0.01430178, -0.04183309, ...,  0.01190011,
        -0.02923723, -0.01942215],
       [ 0.06207929,  0.00980645, -0.04013105, ..., -0.00050627,
         0.05970911, -0.01466273]], dtype=float32)

In [4]:
Location

array([[1942139, 1944305, 1935800, 1936218, 1941286, 1939921, 1936984,
        1942443, 1926429, 1943996]])

In [12]:
from database import engine
# Define your query
query = 'SELECT "Id", "Title", "Description" FROM "Jobs"'

# Read the data into a DataFrame
df = pd.read_sql(query, engine)



In [13]:
Location = Location.reshape(-1)
# Filter the DataFrame where 'Category' is in the list
filtered_df = df[df['Id'].isin(Location)]
filtered_df.head(10)

,Id,Title,Description
32,1935800,UX Frontend Developer,"<div class='""content-intro""'><p><strong>We Mak..."
76,1936218,FullStack Developer,"<div class='""content-intro""'><p><strong>We Mak..."
1271,1926429,Paid Media Specialist,"<div class=""h3""><strong> Fully remote for LATA..."
2205,1939921,Software Engineer,"<p><strong>Ar you based in Argentina, Brazil o..."
2475,1936984,Technical Project Manager,"<p>We are Kubikware™, a digital agency with <s..."
2599,1941286,Python Web Developer,<br/><br/><div>Company Description</div><p><st...
2768,1942443,Mobile UX/UI Designer,<div><p><strong>We Make Remote Work Remarkable...
2855,1942139,Data Analyst,<div><strong>Join Our Team</strong></div><div>...
3036,1944305,Data Insights Analyst,<div><b>Join Our Team</b></div><div><br/></div...
3194,1943996,Senior Frontend Developer,<div>Your responsibilities</div><p>We are look...
